In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import rcParams
import datetime as dt
import sqlalchemy as db

In [2]:
#Set Fonts/Plot Style
plt.style.use('ggplot')
rcParams['figure.figsize'] = [12.0, 10.0]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 100
rcParams['font.size'] = 20
rcParams['legend.fontsize'] = 'large'
rcParams['figure.titlesize'] = 'medium'
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['xtick.direction'] = 'in'
rcParams['ytick.direction'] = 'in'
rcParams['xtick.top'] = True
rcParams['ytick.right'] = True
rcParams['xtick.major.size'] = 10
rcParams['ytick.major.size'] = 10
rcParams['xtick.minor.size'] = 5
rcParams['ytick.minor.size'] = 5
rcParams['xtick.minor.visible'] = True
rcParams['ytick.minor.visible'] = True

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, and_, or_

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn=engine.connect()

In [5]:
# Use the Inspector to explore the database and print the table names
inspector=inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [6]:
# Use Inspector to print the column names and types
columns=inspector.get_columns('measurement')
for column in columns:
    print(column['name'], column['type'])

print('*'*50)
columns=inspector.get_columns('station')
for column in columns:
    print(column['name'], column['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT
**************************************************
id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [7]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# reflect the tables
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Use `engine.execute` to select and display the first 10 rows from the measurement table
data_measurement=engine.execute("SELECT * FROM measurement")
df_measurement=pd.DataFrame(data_measurement)
df_measurement.head(10)

# also can use
#engine.execute("SELECT * FROM measurement LIMIT 10").fetchall()

,0,1,2,3,4
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0
5,6,USC00519397,2010-01-07,0.06,70.0
6,7,USC00519397,2010-01-08,0.00,64.0
7,8,USC00519397,2010-01-09,0.00,68.0
8,9,USC00519397,2010-01-10,0.00,73.0
9,10,USC00519397,2010-01-11,0.01,64.0


In [10]:
# Use `engine.execute` to select and display the first 10 rows from the station table
data_station=engine.execute("SELECT * FROM measurement")
df_station=pd.DataFrame(data_station)
df_station.head(10)

# also can use
#engine.execute("SELECT * FROM station LIMIT 10").fetchall()

,0,1,2,3,4
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0
5,6,USC00519397,2010-01-07,0.06,70.0
6,7,USC00519397,2010-01-08,0.00,64.0
7,8,USC00519397,2010-01-09,0.00,68.0
8,9,USC00519397,2010-01-10,0.00,73.0
9,10,USC00519397,2010-01-11,0.01,64.0


In [11]:
#using pd.read_sql to select and display (also able to see columns name)
df_measurement=pd.read_sql("SELECT * FROM measurement", conn)
df_station=pd.read_sql("SELECT * FROM station", conn)

df_station.head(10)

,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,4,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,6,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,7,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,8,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,9,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [12]:
df_measurement.tail()

,id,station,date,prcp,tobs
19545,19546,USC00516128,2017-08-19,0.09,71.0
19546,19547,USC00516128,2017-08-20,NaN,78.0
19547,19548,USC00516128,2017-08-21,0.56,76.0
19548,19549,USC00516128,2017-08-22,0.50,76.0
19549,19550,USC00516128,2017-08-23,0.45,76.0


In [13]:
# Find the most recent date in the data set.

In [14]:
#most recent date in the data set
recent_date=session.query(Measurement).filter(Measurement.id.between(19000,19550))
#for row in recent_date:
    #print(row.date)

In [15]:
#most recent date in the data set
recent_date=session.query(Measurement).filter(or_(
                                       Measurement.date.like('2016%'),
                                       Measurement.date.like('2017%'))
                                             )
#for row in recent_date:
    #print(row.date)

In [21]:
#Selecting only the date and prcp values.
results=session.query(Measurement.date, Measurement.prcp).filter(or_(
                                       Measurement.date.like('2016%'),
                                       Measurement.date.like('2017%'))
                                             )

In [22]:
# Save the query results as a Pandas DataFrame
df=pd.DataFrame(results, columns=['Date', 'Prcp'])
df

,Date,Prcp
0,2016-01-01,0.00
1,2016-01-02,0.00
2,2016-01-03,0.00
3,2016-01-04,0.00
4,2016-01-05,0.00
...,...,...
3701,2017-08-19,0.09
3702,2017-08-20,NaN
3703,2017-08-21,0.56
3704,2017-08-22,0.50


In [18]:
# Use Pandas `read_sql_query` to load a query statement directly into the DataFrame
stmt=session.query(Measurement.date, Measurement.prcp).filter(or_(
                                       Measurement.date.like('2016%'),
                                       Measurement.date.like('2017%'))
                                             ).statement
df_stat=pd.read_sql_query(stmt, session.bind)
df_stat.head()

,date,prcp
0,2016-01-01,0.0
1,2016-01-02,0.0
2,2016-01-03,0.0
3,2016-01-04,0.0
4,2016-01-05,0.0


In [19]:
#set the index to the date column
df_stat.set_index('date', inplace=True)
df_stat

,prcp
date,
2016-01-01,0.00
2016-01-02,0.00
2016-01-03,0.00
2016-01-04,0.00
2016-01-05,0.00
...,...
2017-08-19,0.09
2017-08-20,NaN
2017-08-21,0.56


In [ ]:
# Sort the dataframe by date
df_stat.sort_index()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
fig, ax=plt.subplots(figsize=(15,15))
df_stat.plot(kind='bar', 'prcp' )

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df_stat.describe()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()